## Import

In [6]:
import pandas as pd
import talib
import backtrader as bt
from dataclasses import dataclass, asdict

## extract

In [7]:
daily: pd.DataFrame = pd.read_excel('gbp_usd_1D.xlsx')
h1: pd.DataFrame = pd.read_excel('gbp_usd_1h.xlsx')



## transform

### definitions

In [8]:
@dataclass
class TradeSignal:
    order_type: str
    entry_price: float
    sl: float
    tp0: float
    tp1: float
    tp2: float


def set_candle_type(r):
    ratio = r['true_range'] / r['atr']

    if ratio < 0.8:
        return 'spinning'
    if 1.2 > ratio >= 0.8:
        return 'standard'
    if 2.4 > ratio >= 1.2:
        return 'long'
    if ratio >= 2.4:
        return 'spike'


def get_candle_type_id(candle_type):
    if candle_type == 'spinning':
        return 0
    if candle_type == 'standard':
        return 1
    if candle_type == 'long':
        return 2
    if candle_type == 'spike':
        return 3


def candle_color(r):
    if r['open'] > r['close']:
        return 'r'
    if r['open'] < r['close']:
        return 'g'

    return 'y'


def set_ma_status(row):
    if row['close'] > row['ma']:
        return 'upper'
    if row['close'] < row['ma']:
        return 'under'
    return 'eq'




### indicators

In [9]:
h1['atr'] = talib.ATR(
    close=h1['close'],
    high=h1['high'],
    low=h1['low'],
    timeperiod=24
)
h1['rsi'] = talib.RSI(
    real=h1['close'],
    timeperiod=9
)
h1['ma'] = talib.MA(
    real=h1['close'],
    timeperiod=24
)
h1['true_range'] = talib.TRANGE(
    close=h1['close'],
    high=h1['high'],
    low=h1['low'],
)


### info

In [10]:
h1['range'] = h1['high'] - h1['low']
h1['candle_color'] = h1.apply(candle_color, axis=1)
h1['candle_type'] = h1.apply(set_candle_type, axis=1)
h1['candle_type_id'] = h1['candle_type'].apply(get_candle_type_id)
h1['ma_status'] = h1.apply(set_ma_status, axis=1)
h1['datetime'] = pd.to_datetime(h1['datetime'])
h1['signal'] = pd.NA


### drop Nones

In [11]:
h1 = h1.iloc[24:]
h1 = h1.drop(columns=['Unnamed: 0'])
h1 = h1.reset_index(drop=True)


### add signals

In [12]:

ma_status = h1.at[0, 'ma_status']
for i, element in h1.iterrows():

    if ma_status == element['ma_status']:
        continue

    ma_status = element['ma_status']

    h1.loc[i, 'signal'] = 'buy' if ma_status == 'upper' else 'sell'
h1['signal'] = h1['signal'].fillna(0)
h1['signal_id'] = h1['signal'].apply(lambda s: -1 if s == 'sell' else 1)

In [80]:
class CustomPandasData(bt.feeds.PandasData):
    lines = ('atr', 'ma', 'signal_id', 'candle_type_id', 'rsi')  # add new lines
    params = (
        ('signal_id', -1),
        ('atr', -1),
        ('ma', -1),
        ('candle_type_id', -1),
        ('rsi', -1),
    )


In [81]:
class StrategyA(bt.Strategy):

    def calculate_volume(
            self,
            entry_price: float,
            stop_price: float,
            balance: int = 10000,
            risk: int = 1,
    ):
        risk_amount = balance * (risk / 100)
        per_unit_risk = abs(entry_price - stop_price)
        units = risk_amount / per_unit_risk
        return int(units)

    def next(self):

        cal_ma_status = lambda close, ma: 'upper' if close > ma else 'under'

        current_ma_status = cal_ma_status(self.data.close[0], self.data.ma[0])
        previous_ma_status = cal_ma_status(self.data.close[-1], self.data.ma[-1])

        if current_ma_status == previous_ma_status:
            return

        if current_ma_status == "upper":
            price = self.data.close[0]

            entry_price = price
            stop_loss = entry_price - self.data.atr[0] * 2.4
            take_profit = entry_price + self.data.atr[0] * 4.8

            self.buy_bracket(
                price=entry_price,
                stopprice=stop_loss,
                limitprice=take_profit,
                size=self.calculate_volume(
                    entry_price,
                    stop_loss,
                    risk=2
                )
            )
        if current_ma_status == "under":
            price = self.data.close[0]

            entry_price = price
            stop_loss = entry_price + self.data.atr[0] * 2.4
            take_profit = entry_price - self.data.atr[0] * 4.8

            self.sell_bracket(
                price=entry_price,
                stopprice=stop_loss,
                limitprice=take_profit,
                size=self.calculate_volume(
                    entry_price,
                    stop_loss,
                    risk=2
                )
            )



In [82]:
data = CustomPandasData(dataname=h1.set_index("datetime"))

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(StrategyA)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade')
cerebro.broker.setcash(10000)
cerebro.broker.setcommission(commission=0, leverage=500)
result = cerebro.run()

In [83]:
cerebro.broker.getvalue()

41040.493430749666

In [84]:
trade_rsult = result[0].analyzers[0].get_analysis()

In [85]:
trade_rsult.keys()

odict_keys(['total', 'streak', 'pnl', 'won', 'lost', 'long', 'short', 'len'])

In [86]:
trade_rsult['short']

AutoOrderedDict([('total', 1066),
                 ('pnl',
                  AutoOrderedDict([('total', 12609.596501570053),
                                   ('average', 11.828889776332133),
                                   ('won',
                                    AutoOrderedDict([('total',
                                                      130863.84587442495),
                                                     ('average',
                                                      389.47573176912186),
                                                     ('max',
                                                      2142.6455624368273)])),
                                   ('lost',
                                    AutoOrderedDict([('total',
                                                      -118254.24937285476),
                                                     ('average',
                                                      -161.99212242856817),
                       